## Build a HybridUrb model from Delft3DFM - example of Eindhoven

This notebook demonstrates how to prepare a **HybridUrb** model from **Delft3D FM** model using an example of Eindhoven.

### Preperation

> **NOTE**: 
All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

Running notebook requires the installation of dependencies which can be done by:

In [ ]:
# This should return Requirement already satisfied if installation is complete.
!pip install hybridurb[examples]

HybridUrb has a main dependency which is **hydromt-delft3dfm**. It provides api access Delft3D FM model files. 
The current example notebook needs newest development of hydromt-delft3dfm v0.2.0.dev0. We can install it from git repository:


In [ ]:
!pip install git+https://github.com/Deltares/hydromt_delft3dfm.git

Check if the HydroMT-Delft3D FM dependency is installed correctly:

In [ ]:
# this should return "dflowfm (hydromt_delft3dfm 0.2.0.dev0)" to build DFlowFM components of Delft3D FM
!hydromt --models

After that checks out, we can start python and load all the neccessary packages:

In [ ]:
import logging
from pathlib import Path
from typing import Optional

import geopandas as gpd
import numpy as np
from pyproj import CRS
import matplotlib.pyplot as plt

import hybridurb.networksutils.delft3dfm_utils as delft3dfm_utils
from hydromt_delft3dfm import DFlowFMModel

### Delft3D FM model for Eindhoven

First, let's have a quick look at the Eindhoven model. 

In [ ]:
# Define the root path where the project files are located
model_root = Path(r"..\data\Eindhoven\Delft3DFM")

# Define the filename of the Delft3D FM model configuration file (*.mdu) as relative path to the model root.
mdu_fn = "dflowfm\FlowFM.mdu"

# read the delft3dfm model (there will be an error message but we can ignore that)
model = DFlowFMModel(root=model_root, config_fn=mdu_fn, mode="r+", crs=28992)

# read the clip region
clip_region = gpd.read_file(r"..\data\Eindhoven\region.geojson")

# make a plot of the model and the region of interest
fig, ax = plt.subplots()
model.geoms["branches"].plot(ax=ax, color="tab:blue", zorder=1)
clip_region.plot(ax=ax, color = 'yellow', alpha = 0.5)

### Model setup configuration

#FIXME
The network model configuration file (YML) contains the model setup configuration and determines which methods are used to prepare the HybridUrb and in which order and optionally sets non-default arguments for each method. We have prepared several example yml-files which are available in the model repository [examples folder](https://github.com/xldeltares/hybridurb/tree/main/examples) and from the [docs (building a model)](https://deltares.github.io/hydromt_delft3dfm/latest/user_guide/dflowfm_build). 

Each section, before indent, (e.g. get_geoms_from_model) corresponds to a model method. All model methods are explained in the [docs (model components)](https://deltares.github.io/hydromt_delft3dfm/latest/user_guide/dflowfm_model_setup.html). The `global` section contains direct model initialisation properties. 

We will load the provided config.yaml file for Eindhoven for inspection:

In [ ]:
fn_yml = Path("Eindhoven/config.yml")
with open(fn_yml, "r") as f:
    txt = f.read()
print(txt)

#FIXME
Looking at this file, you see that we will prepare the following information for our model:

- [global](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.html#hydromt_delft3dfm.DFlowFMModel): Our model will be defined in the projected CRS WGS84 EPSG 3857. TO ensure the 1D network is connected, we will allow snapping of 25 meters using argument network_snap_offset. The 1D computational grid for the open water system will be created at 40 meters distance using argument openwater_computation_node_distance. 
- [get_geoms_from_model](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_rivers_from_dem.html): We will derive the 1D rivers lines based on the *MERIT Hydro DEM* and that intersects with the region bounding box [12.4331, 46.4661, 12.5212, 46.5369]. River roughness will be a Manning constant of 25.0 and we will use data from *rivers_lin2019* database to prepare rectangular cross-sections. 
- [setup_networkmodel](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_pipes.html): We will add pipes by assuming that they are located under the roads, with a standard length of 50 meters. The roads are defined in the *grip_roads* data and the standard length are defined in the argument spacing. Roughness will be a WhiteColeBrook constant of 0.003 and we will use a circular cross-sections of 0.5m. The pipes invert levels will be derived from the *MERIT Hydro DEM* assuming a constant depth of the pipe underground of 2 meters. 

Feel free to use the links of each components to know more about how each function works and what are all the available settings.

> **NOTE**: 
  In this example, we apply region argument in both setup_rivers_from_dem and setup_pipes. They are identical in this example but one can alter them to have different region for rivers and for pipes. This allows the user to customize their model as much as possible, as the extent of the 1D river can differ from the extent of the 1D urban network or the 2D inundation grid. The region bbox are in WGS 84 as required, but the user should specify the dflowfm model destination CRS in the `global` section in order to allow to use grid definition data (rivers, pipes) from different data sources or CRS. 

### Set up a HybridUrb network

In [ ]:
from hybridurb.networksutils.network_from_delft3dfm import Delft3dfmNetworkWrapper
import os
os.chdir(r"c:\Developments\hybridurb\examples")
mywrapper = Delft3dfmNetworkWrapper(fn_yml)
mywrapper.get_geoms_from_model()
mywrapper.get_network_from_geoms()

The example above means the following: run **hydromt build** with:

* `dflowfm` : i.e. build a FM model
* `./build/dflowfm_1d` : output model folder
* `-i dflowfm_build1d.yml`: model build configuration file
* `-d artifact_data`: data catalog to use. Here the default artifact data catalog with global data for Northern Italy.
* `--fo`: force overwritting in case a model already exists in the output folder.
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug and info messages are provided rather than only warnings and errors.

> NOTE: You can use as many data catalogs as you need within the same command line, by repeating the `-d` options and name of the catalog to use several times.

Next we check the model files that have been created. 

In [ ]:
import os

root = "./build/dflowfm_1d/"
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        print(f" - {name}")

The model root should contain three files and four folders:

- dimr_config.xml: Configuration file to execute Delft3D FM in Deltares Integrated Model Runner (DIMR).
- hydromt.log: log saved by HydroMT on model building steps.
- hydromt_data.yml: information saved by HydroMT on used datasets for reproducibility.
- **dflowfm** folder: dflowfm files, here for our 1D model. Contains all necessary information to run a 1D model including the mesh1d, roughness, cross-sections, 1D boundary forcing.   The configuration MDU file is also created as well as a *.gui* file to visualize the 1D network in the Delft3D FM GUI.
- **geoms** folder: folder containing a copy of the dflowfm data in an easily readable GIS format rather than the ini or UGRID netcdf file.
- **maps** folder: folder containing regular gridded data that Delft3D FM can interpolate to the 2D mesh grid. Here empty as we have a 1D model only.

### Visualize and/or inspect model schematization

* We can now load the **dimr_config.xml** into **Delft3D FM GUI**.
* The **dflowfm plot** [example notebook](https://deltares.github.io/hydromt_delft3dfm/latest/_examples/plot_dflowfm_mesh.html) contains scripts to visualize your model in python: 